#### Importing dependancies

In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import seaborn as sns
import matplotlib.pyplot as plt
import re
from tqdm import tqdm
import datetime
from textblob import TextBlob
import requests 
from bs4 import BeautifulSoup
# from sys import
import warnings
warnings.filterwarnings("ignore")

In [2]:
def scrape_yahoo(search_term):
    pn = 1
    l = []
    t = []

    for lkd in range(5):
        url = "https://news.search.yahoo.com/search?q={}&pz=10&b={}".format(search_term, pn)
        response = requests.get(url)
        soup = BeautifulSoup(response.text, "html.parser")
        headlines = soup.find_all("div")

        d = []
        for h in headlines:
            d.append(TextBlob(h.get_text()))


        for i in range(len(d[0].split("...")[:-1])):
            try:
                ind = [[i.start(),i.end()] for i in re.finditer(re.compile("|".join(["\d+\s*(?:year|years)\s*ago",
                                                                                     "\d+\s*(?:day|days)\s*ago",
                                                                                     "\d+\s*(?:hour|hours)\s*ago",
                                                                                     "\d+\s*(?:minute|minute)\s*ago",
                                                                                     "\d+\s*(?:seconds|seconds)\s*ago",
                                                                                     "Local\s*Answers\s*Shopping\s*More"])), 
                                                                d[0].split("...")[i])]
                timings = re.findall(re.compile("|".join(["\d+\s*(?:year|years)\s*ago",
                                                          "\d+\s*(?:day|days)\s*ago",
                                                          "\d+\s*(?:hour|hours)\s*ago",
                                                          "\d+\s*(?:minute|minute)\s*ago",
                                                          "\d+\s*(?:seconds|seconds)\s*ago"])), 
                                                                d[0].split("...")[i])
                if i==0:
                    l.append((d[0].split("...")[i][ind[0][1] : ind[1][0]], timings[0]))
    #                 t.append(timings[0])
                else:
                    l.append((d[0].split("...")[i][:ind[0][0]], timings[0]))
    #                 t.append(timings[0])
            except:
                pass

        pn = pn + 10
        
        
    df = pd.DataFrame(l, columns=["News","Timing"])

    df["Date"] = np.nan
    for i in range(len(df["Timing"])):
        if "day" in df["Timing"][i]:
            df["Date"][i] = datetime.datetime.now()-datetime.timedelta(days=int(re.findall(re.compile("(\d+)"),
                                                                                         df["Timing"][i])[0]))
        elif "hour" in df["Timing"][i]:
            df["Date"][i] = datetime.datetime.now()-datetime.timedelta(hours=int(re.findall(re.compile("(\d+)"),
                                                                                          df["Timing"][i])[0]))

    df = df.sort_values(by=["Date"], ascending=False).reset_index(drop=True)
    vader = SentimentIntensityAnalyzer()
    df["Compounded_Polarity"] = [vader.polarity_scores(k)["compound"] for k in df["News"]]
        
    return df

In [3]:
df = scrape_yahoo("PLUG power")

In [4]:
df

,News,Timing,Date,Compounded_Polarity
0,Stocks That Hit 52-Week Highs On ThursdayBenz...,1 hour ago,2020-11-12 11:05:51.142312,0.0000
1,ROCE Insights For Plug PowerBenzinga·,2 hours ago,2020-11-12 10:05:51.139377,0.0000
2,3 Hydrogen Stocks to Buy for a Fuel Cell Futu...,3 hours ago,2020-11-12 09:05:51.139377,0.0000
3,Why the Huge Valuation of WKHS Stock Isn't Ju...,4 hours ago,2020-11-12 08:05:51.139377,0.3182
4,56 Biggest Movers From YesterdayBenzinga·,7 hours ago,2020-11-12 05:05:51.141369,0.0000
5,"Plug Power: Next Stop, $30?TipRanks via Yahoo...",18 hours ago,2020-11-11 18:05:51.139377,-0.2960
6,Westwater Resources Stock Needs More Than Hyp...,22 hours ago,2020-11-11 14:05:51.140375,0.0000
7,"Short Sellers Pile Into One Alt Energy Stock,...",23 hours ago,2020-11-11 13:05:51.141369,0.2732
8,"Even After Post-Election Rebound, Lordstown S...",23 hours ago,2020-11-11 13:05:51.141369,0.0000
9,This Recent Prevailing Weakness Is Your Chanc...,1 day ago,2020-11-11 12:05:51.140375,-0.2023
